In [17]:
import pandas as pd 
import sys
import json
# sys.path.append('../models/topic_model/')
# import sampler_cy
import os
from sklearn.metrics import cohen_kappa_score

In [2]:
os.listdir('../data/')

['ace_phase2',
 'Annotations.zip',
 'combined_data_df.csv',
 'finlayson_anno_offsets_new.json',
 'finlayson_data.csv',
 'glove.6B.zip',
 'LDC2003T11.tgz',
 'my_checked_annotations.csv',
 'nyt_sample.csv',
 'ruihong_news_structure_data',
 'text_df.csv',
 'XML']

In [18]:
pd.read_csv('../data/my_checked_annotations.csv', index_col=0).pipe(lambda df: 
    cohen_kappa_score(df['my_updated_labels'], df['finlayson_labels'])
)

0.765489404641776

In [20]:
pd.read_csv('../data/combined_data_df.csv')['label'].value_counts()

VERB    674
CIRC    349
MAIN    277
EVAL    162
CONS    157
PREV    131
LEAD     91
HIST     90
EXPE     61
EXPL      5
SECD      4
Name: label, dtype: int64

# Format New Data

In [61]:
os.listdir('../data/ace_phase2/data/ace2_train/')

['bnews', 'npaper', 'nwire']

In [150]:
corpus = json.load(open('../app/data/untagged_records.json'))
corpus = pd.DataFrame(corpus).assign(doc_key=lambda df: df['doc_key'].str.replace('.sgm', ''))

In [81]:
tagged_data['doc_key'].unique().shape

(119,)

In [20]:
import glob
import spacy
import langdetect
from langdetect.lang_detect_exception import LangDetectException
def lang(x):
    try:
        return langdetect.detect(x)
    except LangDetectException:
        return 'na'

nlp = spacy.load('en_core_web_sm')

In [153]:
key, t2 = corpus.iloc[-5]
sent_s = pd.Series([s.text for s in nlp(t2.replace('\n', ' ')).sents])

In [165]:
(sent_s
 .str.strip()
 .to_frame('text')
 .loc[lambda df: df['text'].apply(lang) == 'en']
 .loc[lambda df: ~df['text'].str.contains(key)]
)

,text
4,By MATTHEW L. WALD &LR; &QC; &QL; &UR; c.199...
5,"The auditors, from the General Accounting Offi..."
6,"At the aviation agency, the investigators foun..."
7,The findings were the subject of a pair of GAO...
8,"In response, a State Department spokesman, Jam..."
9,"The aviation agency, in contrast, said in a st..."
11,"The agency's computers are aging, and the stat..."
12,"The auditors' report, however, said that ``whi..."
13,And they are certainly vulnerable to attack ``...
14,While the usual mode of hacking is over the In...


In [7]:
from xml.etree import ElementTree as ET
from bs4 import BeautifulSoup
import re
import glob
import pandas as pd 

In [8]:
gigaword_corpus = []
for article_file in glob.glob('../data/ruihong_news_structure_data/sampled/*/*'):
    t2 = open(article_file).read().strip()
    soup = BeautifulSoup(t2)
    article_type = re.findall('type=\"(.*?)\"', t2)[0]
#     article_class = re.findall('class=\"(.*?)\"', t2)[0]
    article_id = re.findall('id=\"(.*?)\"', t2)[0]
    paragraphs = list(map(lambda x: x.text.strip().replace('\n', ' '), soup.find_all('p')))
    
    for p in paragraphs:
        gigaword_corpus.append({'article_id': article_id, 'text': p})

In [9]:
gigaword_corpus_paragraphs = pd.DataFrame(gigaword_corpus)

In [11]:
pd.read_csv('../data/combined_data_df.csv').shape

(2004, 6)

In [12]:
gigaword_corpus_paragraphs.shape

(17811, 2)

In [276]:
nyt_corpus = pd.read_csv('../../newspaper-pages/nyt_corpus/unprocessed_full_data_df.csv', index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [321]:
nyt_sample = (nyt_corpus
 .loc[lambda df: df['print_section_and_page'] == 'A-001']
 .loc[lambda df: df['body'].notnull()]
 [['id', 'body']].sample(1000)
)

In [295]:
nyt_sample.to_csv('../data/nyt_sample.csv')

In [13]:
nyt_sample = pd.read_csv('../data/nyt_sample.csv')

In [19]:
from more_itertools import unique_everseen
from tqdm import tqdm_notebook as tqdm

In [ ]:
nyt_sentences = []
for _, _, doc_id, text in tqdm(nyt_sample.itertuples(), total=len(nyt_sample)):
    text = text.replace('\n', ' ')
    doc = nlp(text)
    sents = [x.text for x in doc.sents]
    sents = list(unique_everseen(sents))
    for s in sents:
        nyt_sentences.append({'id': doc_id, 'text': s})

In [329]:
nyt_sentences_corpus = pd.DataFrame(nyt_sentences).loc[lambda df: df['text'].str.strip().str.len() > 9]

# Combine Data For Topic Model

In [ ]:
nyt_sentences_corpus.head()

In [333]:
gigaword_corpus_paragraphs.head()

,article_id,text
0,NYT_ENG_20100108.0149,It's official: China is the world's largest ve...
1,NYT_ENG_20100108.0149,Passenger car sales totaled 10.3 million in Ch...
2,NYT_ENG_20100108.0149,"By contrast, U.S. car and light truck sales sh..."
3,NYT_ENG_20100108.0149,Demand for vehicles picked up in the U.S. in t...
4,NYT_ENG_20100108.0149,Chinese auto sales began to take off early in ...


In [336]:
tagged_ace_df = pd.read_csv('../data/combined_data_df.csv')[['doc_key', 'paragraph_text', 'label']]

In [342]:
text_df = pd.concat([
    tagged_ace_df.rename(columns={'paragraph_text':'text'}),
    gigaword_corpus_paragraphs.rename(columns={'article_id': 'doc_key'}),
    nyt_sentences_corpus.rename(columns={'id': 'doc_key'})
], sort=False)

In [344]:
text_df.to_csv('../data/text_df.csv')

In [489]:
text_df.shape

(72631, 5)

In [554]:
import numpy as np 
import string
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=25, max_df=.5, stop_words='english', lowercase=True)

In [555]:
cv.fit(text_df['text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.5, max_features=None, min_df=25,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [556]:
vocab = cv.vocabulary_

In [557]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [558]:
def process_paragraph(text):
    for p in string.punctuation:
        text = text.replace(p, ' ')
    output = []
    for t in text.split():
        if t in vocab:
            output.append(int(vocab.get(t)))
    return output

In [559]:
text_df['vectors'] = text_df['text'].str.lower().apply(process_paragraph)

In [560]:
label_mapper = {
 'LEAD': 0,
 'CIRC': 1,
 'MAIN': 2,
 'VERB': 3,
 'CONS': 4,
 'PREV': 5,
 'EVAL': 6,
 'HIST': 7,
 'EXPE': 8,
 'SECD': 9,
 'EXPL': 10,
 np.nan: -1
}

In [561]:
text_df['label_id'] = text_df['label'].map(label_mapper)

In [562]:
doc_df = text_df.groupby('doc_key')[['vectors', 'label_id']].aggregate(list)

In [563]:
doc_df['has_labels'] = doc_df['label_id'].apply(lambda x: not all(map(lambda y: y==-1, x)))

In [564]:
with open('../models/topic_model/labeled_data/index.txt', 'w') as f:
    for idx in list(doc_df.index):
        f.write(idx)
        f.write('\n')

In [565]:
jsons = doc_df.rename(columns={'vectors':'paragraphs', 'label_id':'labels'}).to_dict(orient='records')

In [566]:
with open('../models/topic_model/labeled_data/doc_vecs.json', 'w') as f:
    for j in jsons:
        f.write(json.dumps(j, default=convert))
        f.write('\n')

In [567]:
with open('../models/topic_model/labeled_data/vocab.txt', 'w') as f:
    for word in pd.Series(cv.vocabulary_).sort_values().index:
        f.write(word)
        f.write('\n')

In [568]:
del sampler_cy

In [524]:
## check

In [525]:
with open('../models/topic_model/labeled_data/doc_vecs.json') as f:
    for line in f:
        t = json.loads(line)
        break

# Process Data for BiLSTM

In [1]:
import pandas as pd 
import random
from sklearn.model_selection import train_test_split

In [50]:
data_df = pd.read_csv('../data/combined_data_df.csv')

In [51]:
data_df = (data_df
 .loc[lambda df: df['label'].notnull()]
 .loc[lambda df: df['label'].isin(df['label'].value_counts()[:-2].index)]
)

In [52]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [55]:
from tqdm import tqdm_notebook as tqdm

In [59]:
def process_record(record):
    output = []
    doc_key = record['doc_key']
    label = record['label']
    sents = list(nlp(' '.join(record['paragraph_text'].split())).sents)
    for s in sents:
        output.append({
            'doc_key': doc_key,
            'label': label,
            'paragraph_text': s.text
        })
    return output

In [60]:
all_records = []
for row in tqdm(data_df.to_dict(orient='records')):
    all_records += process_record(row)

In [70]:
final_df = (
    pd.DataFrame(all_records)
        .loc[lambda df: df['paragraph_text'].str.len() > 5]
)

In [71]:
grouped = (final_df.groupby('doc_key')[['paragraph_text', 'label']].aggregate(list))

In [72]:
idx_train, idx_test = train_test_split(grouped.index, test_size=0.1)
idx_train, idx_val = train_test_split(idx_train, test_size=0.2, random_state=1)

In [73]:
len(idx_train)

85

In [74]:
with open('../models/bilstm/data/train_keys.txt', 'w') as train_keys, \
open('../models/bilstm/data/train_keys.txt', 'w') as test_keys, \
open('../models/bilstm/data/train_keys.txt', 'w') as val_keys, \
open('../models/bilstm/data/train_input.txt', 'w') as train_f, \
open('../models/bilstm/data/test_input.txt', 'w') as test_f, \
open('../models/bilstm/data/val_input.txt', 'w') as val_f:
    for doc_key, paragraphs, labels in grouped.itertuples():
        if doc_key in idx_train:
            f = train_f
            keys = train_keys
        elif doc_key in idx_test:
            f = test_f
            keys = test_keys
        elif doc_key in idx_val:
            f = val_f
            keys = val_keys
        else:
            print('not found...')

        for par, label in zip(paragraphs, labels):
            f.write('%s\t%s\n' % (' '.join(par.split()), label))
        f.write('\n')
        keys.write(doc_key)
        keys.write('\n')

In [1]:
from keras_bert import get_pretrained, PretrainedList, get_checkpoint_paths

Using TensorFlow backend.


In [2]:
PretrainedList.wwm_uncased_large

'https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip'

In [3]:
model_path = get_pretrained(PretrainedList.wwm_uncased_large)
paths = get_checkpoint_paths(model_path)
print(paths.config, paths.checkpoint, paths.vocab)

C:\Users\alexa\.keras\datasets\wwm_uncased_L-24_H-1024_A-16\bert_config.json C:\Users\alexa\.keras\datasets\wwm_uncased_L-24_H-1024_A-16\bert_model.ckpt C:\Users\alexa\.keras\datasets\wwm_uncased_L-24_H-1024_A-16\vocab.txt
